# Photo Spec Calculator
This tool is used to calculate unknown photo specs (pixel size and DPI or photo size) from known specs (DPI or photo size).

In [46]:
import os
from PIL import Image
import statistics

# Override PIL max pixel limit
Image.MAX_IMAGE_PIXELS = None

In [63]:
def calculate_photo_specs(path, dpi=None, photo_size=None, dpi_tolerance=0.01, verbose=False):
    """
    Pass a photo path and either DPI or photo size to calculate 
    pixel size and the other parameter. 
    @param {str} path The path to the photo image.
    @param {int} dpi The scanning resolution of the image in dots
    per inch. If DPI is not provided, it will be calculated based on
    photo size. DPI is calculated as the mean of the length and width
    DPI. 
    @param {tuple of ints} photo_size The physical size of the scanned 
    image in millimeters format (length, width). If photo size is not
    provided, it will be calculated based on DPI.
    @param {float, default 0.5} dpi_tolerance: The maximum proportional
    difference between calculated length and width DPI. If the provided photo
    size is out of proportion to the image size in pixels beyond this threshold,
    (for example, if length and width were switched in a rectangular image), an
    error will be raised.
    @return {tuple} Photo specs in the format ((pixel length in mm, pixel width in mm), dpi, (length in mm, width in mm))
    """
    if all((dpi, photo_size)):
        raise Exception("Provide only dpi or photo_size argument, not both.")
    
    img = Image.open(path)
    l, w = img.size
    
    if dpi:
        calculating = "Photo size"
        l_in = l / dpi
        w_in = w / dpi
        
        l_mm = l_in * 25.4
        w_mm = w_in * 25.4
        
        photo_size = (l_mm, w_mm)
    
    elif photo_size:
        calculating = "DPI"
        l_mm = photo_size[0]
        w_mm = photo_size[1]
        
        l_in = l_mm / 25.4
        w_in = w_mm / 25.4
        
        l_dpi = l / l_in
        w_dpi = w / w_in
        
        dpi_mismatch = abs(l_dpi / w_dpi - 1)
        dpi_mismatch_print = str(round(dpi_mismatch * 100, 2))
        
        if dpi_mismatch > dpi_tolerance:
            raise Warning(f"""The provided photo_size proportion does not match 
            the image resolution ({dpi_mismatch_print}% mismatch). Confirm that 
            photo_size is accurate. To supress this error, raise the dpi_tolerance
            beyond the current mismatch level.""")
        
        dpi = statistics.mean((l_dpi, w_dpi))
    
    else:
        raise Exception("Provide either dpi or photo_size argument.")
        
    l_pixel = l_mm / l
    w_pixel = w_mm / w
    pixel_size = (l_pixel, w_pixel)
    
    if verbose:
        print(f"""
        Image resolution (px): {l} (L) x {w} (W)
        Calculated: {calculating}\n
        DPI: {dpi}
        DPI mismatch: {dpi_mismatch_print + "%" if calculating == "DPI" else "NA"}
        Photo length (mm): {photo_size[1]}
        Photo width (mm): {photo_size[0]}
        Pixel length (mm): {l_pixel}
        Pixel width (mm): {w_pixel}
        """)
    
    return(pixel_size, dpi, photo_size)

## Example

In [67]:
photo_path = os.path.join("E:", "ORISE", "Projects", "other", "naficy_photogrammetry", "DWW 1965-69 Six Mile GYE", "DWW_3FF_117.tif")

pixel_size, dpi, _ = calculate_photo_specs(photo_path, photo_size=(224, 224), verbose=True)


        Image resolution (px): 18859 (L) x 18862 (W)
        Calculated: DPI

        DPI: 2138.645982142857
        DPI mismatch: 0.02%
        Photo length (mm): 224
        Photo width (mm): 224
        Pixel length (mm): 0.011877618113367622
        Pixel width (mm): 0.011875728978899375
        
